<a href="https://colab.research.google.com/github/aaronmat1905/the-tokenized-tech/blob/main/%5CLLM%20Engineering%5CAgentic%20AI%5CAgentsByHand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The Tokenized Tech**\
**Article 1:** Agents By Hand

In [3]:
# !pip install -q dotenv

In [2]:
# Code for Google-colab platforms
from google.colab import userdata
api_key = userdata.get("GeminiAPI")

**Gemini Client** \
*Custom Implementation*

In [4]:
# Custom Gemini Client:
import os
import requests

# from dotenv import load_env
# api_key = os.getenviron("GEMINI_API_KEY")

class GeminiClient:
	def __init__(self, api_key: str):
		self.api_key = api_key
		self.endpoint = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"
		self.headers = {
			"x-goog-api-key": self.api_key,
			"Content-Type": "application/json"
			}
	def generate_content(self, prompt) -> str:
		payload = {
			"contents": [
					{ "parts":[{"text":prompt}]}
				]
		}
		response = requests.post(self.endpoint, headers= self.headers, json = payload)
		if response.status_code ==200:
			data = response.json()
			try:
				return data['candidates'][0]['content']['parts'][0]['text']
			except (KeyError, IndexError):
				return "No response text found"
		else:
			return f"Error: {response.status_code} - {response.text}"

In [5]:
# Useage of our custom client:
client = GeminiClient(api_key)
# Sample Prompt:
prompt = "Explain what are AI Agents in a sentence."
response = client.generate_content(prompt)
print(f"Response: {response}")

Response: AI Agents are autonomous systems that perceive their environment, make decisions, and act to achieve specific goals.


**Defining Our Tools**

In [9]:
# Tool Registry:
class Tool:
  def __init__(self):
    self.tools = {}
  def registerTool(self, name=None, description=None):
    "Records a Tool"
    def decorator(func):
      tool_name = name or func.__name__
      self.tools[tool_name] = {
          "function_name":func,
          "description":description or func.__doc__
      }
      return func
    return decorator

  def call(self, tool_name:str, *args,**kwargs):
    if tool_name not in self.tools:
      raise ValueError(f"Tool '{tool_name}' does not exist.")
    return self.tools[tool_name]["function"](*args, **kwargs)
  def list_tools(self):
    return {name:meta["description"] for name, meta in self.tools.items()}

In [10]:
# Creating the Tool Registry:
toolFamily = Tool()

**Problem Statement:** Create a Financial Agent

We are going to implement the given 5 functions for our finance agent

\

**EMI**\
`calculate_emi(principal, annual_rate, tenure_years)`

**Purpose:** Calculates monthly EMI for a loan.
**Why it’s useful:** Loan comparison, affordability checks, credit advisory.

---

**CAGR**\
`calculate_cagr(initial_value, final_value, years)`

**Purpose:** Computes Compound Annual Growth Rate for an investment.
**Why it’s useful:** Investment performance metric used in wealth advisory, fintech portfolios.

---
**ROI**\
`calculate_roi(cost, return_value)`

**Purpose:** Calculates Return on Investment (ROI) in percentage.
**Why it’s useful:** Quick performance check for any business or personal financial decision.

---
**Credit Utilization**\
`credit_utilization(total_limit, current_balance)`

**Purpose:** Computes credit utilization ratio.
**Why it’s useful:** Important metric in credit scoring (e.g., 30% threshold rule); useful for credit card users and financial health monitoring.

---
**Retirement Goal Planner**\

`retirement_goal_planner(current_age, retirement_age, desired_monthly_income, annual_rate)`

**Purpose:** Estimates how much one needs to save monthly to reach retirement goals.
**Why it’s useful:** Long-term planning tool—great for banks, insurers, or robo-advisors.



In [ ]:
# @toolFamily.registerTool()
# def sip_returns(monthly_investment: float, years:int,annual_rate: float)-> str:
#   n = years * 12
#   r = annual_rate / (12*100)
#   future_value = monthly_investment * (((1 + r)**n - 1) / r) * (1 + r)
#   return f"Future Value of SIP: ₹{round(future_value, 2)}"